# Data cleaning

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from healthy_candies.load import load_data

%matplotlib

Using matplotlib backend: TkAgg


Here we are interested in the following columns (in fact many columns related to nutri facts are already very sparse, we focus here only on the major ones):

In [2]:
COLS = ['image_small_url', 'code', 'energy_100g', 'fat_100g',
        'saturated-fat_100g', 'carbohydrates_100g', 'sugars_100g',
        'proteins_100g', 'salt_100g', 'sodium_100g',
        'nutrition-score-fr_100g']

In [3]:
df = load_data(COLS, limit_have_nutri_score=False)
df.shape

(698477, 11)

The point of this pre-processing is to have also information on the product image.
So we keep rows only where this feature is available.

In [4]:
urls = df.image_small_url.fillna('')
df = df[urls != '']
df.count()

image_small_url            91644
code                       91644
energy_100g                65436
fat_100g                   64712
saturated-fat_100g         62871
carbohydrates_100g         64457
sugars_100g                63140
proteins_100g              65147
salt_100g                  62583
sodium_100g                62577
nutrition-score-fr_100g    51241
dtype: int64

Finally we keep only the rows for which we have all the information. At the end we have $\approx 51\, 000$ rows.

In [5]:
df_clean = df.dropna()
df_clean.describe()

,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g,nutrition-score-fr_100g
count,50627.000000,50627.000000,50627.000000,50627.000000,50627.000000,50627.000000,50627.000000,50627.000000,50627.000000
mean,1128.073852,14.066029,5.639722,29.155872,12.926757,7.820781,2.479840,0.976223,8.585162
std,1602.410003,83.570264,40.444734,263.626440,37.789098,42.533754,58.061692,22.852254,9.055507
min,0.000000,0.000000,0.000000,0.000000,-0.100000,0.000000,0.000000,0.000000,-14.000000
25%,404.000000,1.100000,0.210000,4.000000,1.000000,1.800000,0.080000,0.031496,1.000000
50%,1029.000000,6.800000,1.900000,14.000000,4.000000,5.900000,0.508000,0.200000,9.000000
75%,1671.000000,22.000000,7.500000,53.500000,16.000000,10.700000,1.220000,0.480000,15.000000
max,300000.000000,18400.000000,8500.000000,57400.000000,6670.000000,9400.000000,6728.460000,2649.000000,40.000000


We can see above that many columns have absurd values, let's have a look at the $2.5$ and $97.5$ percentiles of each columns.

In [6]:
q = df_clean.quantile([0.025, 0.975])
q

,energy_100g,fat_100g,saturated-fat_100g,carbohydrates_100g,sugars_100g,proteins_100g,salt_100g,sodium_100g,nutrition-score-fr_100g
0.025,69.0,0.000,0.0,0.0,0.0,0.0,0.0000,0.00,-6.0
0.975,2736.7,62.705,24.0,82.0,61.7,26.4,4.7752,1.88,26.0


We can see above that the values above are more probable. We will therefore keep the products for which all their properties fit in the acceptable range. This is a simple way to remove outliers quickly.
*NB: we don't do the cleaning for the nutrition score column. Its values are correct.*

In [7]:
# remove crazy values
for col in q.columns:
    if col != "nutrition-score-fr_100g":
        df_clean = df_clean[(df_clean[col] >= q.loc[0.025, col]) & (
            df_clean[col] <= q.loc[0.975, col])]

In [8]:
df_clean.shape

(43454, 11)

At the end we keep $\approx 80\%$ of the processable data.